# Import Libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge geocoder --yes 
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


bran

# Scrap from wikipedia

In [6]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Mexico_City").text

In [8]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [9]:
# create a list to store neighborhood data
neighborhoodList = []

In [10]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [11]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Barrios Mágicos of Mexico City
1,Colonia Algarín
2,Colonia Ampliación Asturias
3,Colonia Asturias
4,Colonia Atlampa


In [12]:
# print the number of rows of the dataframe
kl_df.shape

(80, 1)

In [20]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mexico City, Mexico'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [21]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [22]:
coords

[[19.337675982514867, -99.17849613558197],
 [19.406970000000058, -99.13964999999996],
 [19.407620000000065, -99.13468999999998],
 [19.397529168922603, -99.14007798787453],
 [19.457220000000063, -99.15788999999995],
 [19.442708999999997, -99.10068749999999],
 [19.422300000000064, -99.11851999999999],
 [19.39098000000007, -99.25511999999998],
 [19.328920268318736, -99.28681805276116],
 [19.24384027631671, -99.0187733162589],
 [19.431940000000054, -99.13314999999994],
 [19.371800000000064, -99.15724999999998],
 [19.355360000000076, -99.15082999999998],
 [19.409440000000075, -99.19464999999997],
 [19.33763894009245, -99.20141153052967],
 [19.427526000000004, -99.139851],
 [19.458510000000047, -99.13772999999998],
 [19.512247500000008, -99.08977499999999],
 [19.382044443777367, -99.22044418747298],
 [19.327665792147325, -99.28263552834012],
 [19.336788700582073, -99.21912837645364],
 [19.163368555816117, -99.1605231953002],
 [19.41948000000008, -99.14110999999997],
 [19.413040000000024, -99

In [23]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [24]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [25]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(80, 3)


,Neighborhood,Latitude,Longitude
0,Barrios Mágicos of Mexico City,19.337676,-99.178496
1,Colonia Algarín,19.406970,-99.139650
2,Colonia Ampliación Asturias,19.407620,-99.134690
3,Colonia Asturias,19.397529,-99.140078
4,Colonia Atlampa,19.457220,-99.157890
5,Barrio Chino (Mexico City),19.442709,-99.100687
6,"Benito Juárez, Mexico City",19.422300,-99.118520
7,Bosques de las Lomas,19.390980,-99.255120
8,Colonia Buenavista,19.328920,-99.286818
9,Colonia Buenos Aires,19.243840,-99.018773


In [26]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

# Create a map of Mexico City with neighborhoods superimposed on top

In [27]:
# get the coordinates of  Mexico city
address = 'Mexico City, Mexico'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Mexico city, Mexico {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Mexico city, Mexico 19.4326009, -99.1333416.


In [28]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [29]:
# save the map as HTML file
map_kl.save('map_kl.html')

# Use the Foursquare API to explore the neighborhoods

In [30]:
# define Foursquare Credentials and Version
CLIENT_ID = print('Your credentails')
 # your Foursquare ID
CLIENT_SECRET = ('CLIENT_ID ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2M4G3L040H55PMKRSJDWZGO0XICZRHHZXHSA5F0XU4HI3UKV
CLIENT_SECRET:UEUFNYRA4OLNAYEDDZUZVHAIO44L0XYWO1ZNWL11T2LB135C


# Now, let's get the top 50 venues that are within a radius of 2000 meters

In [31]:
radius = 2000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [32]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3650, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Barrios Mágicos of Mexico City,19.337676,-99.178496,Tennis Express,19.336080,-99.178366,Sporting Goods Shop
1,Barrios Mágicos of Mexico City,19.337676,-99.178496,G Martell College of Music Technology & Audio,19.335261,-99.177563,Music School
2,Barrios Mágicos of Mexico City,19.337676,-99.178496,Teatro G. Martell,19.335169,-99.177254,Public Art
3,Barrios Mágicos of Mexico City,19.337676,-99.178496,Monte Kailas,19.334638,-99.178118,Vegetarian / Vegan Restaurant
4,Barrios Mágicos of Mexico City,19.337676,-99.178496,Café y Té Quiero,19.335079,-99.179744,Breakfast Spot


In [33]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Barrio Chino (Mexico City),50,50,50,50,50,50
Barrios Mágicos of Mexico City,50,50,50,50,50,50
"Benito Juárez, Mexico City",50,50,50,50,50,50
Bosques de las Lomas,50,50,50,50,50,50
Centro (Mexico City),50,50,50,50,50,50
Centro Urbano Benito Juárez,50,50,50,50,50,50
Churubusco,50,50,50,50,50,50
Colonia Algarín,50,50,50,50,50,50
Colonia Ampliación Asturias,50,50,50,50,50,50


# Let's find out how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 280 uniques categories.


In [35]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Sporting Goods Shop', 'Music School', 'Public Art',
       'Vegetarian / Vegan Restaurant', 'Breakfast Spot', 'Taco Place',
       'Japanese Restaurant', 'Café', 'Performing Arts Venue',
       'Argentinian Restaurant', 'Mexican Restaurant', 'Shopping Mall',
       'Snack Place', 'Pet Store', 'Jewelry Store', 'Multiplex',
       'Cosmetics Shop', 'Bookstore', 'Ice Cream Shop', 'Clothing Store',
       'Boutique', 'Park', 'Burger Joint', 'Discount Store',
       'Coffee Shop', 'Restaurant', 'Creperie', 'American Restaurant',
       'Baby Store', "Women's Store", 'Gym / Fitness Center',
       'Accessories Store', 'Health & Beauty Service',
       'Italian Restaurant', 'Brewery', 'Bakery', 'Motorcycle Shop',
       'Steakhouse', 'Pie Shop', 'Seafood Restaurant', 'Gym', 'Diner',
       'Juice Bar', 'Fast Food Restaurant', 'Martial Arts Dojo',
       'Beer Bar', 'Nail Salon', 'Dessert Shop', 'Sushi Restaurant',
       'Bistro'], dtype=object)

In [39]:
# check if the results contain "Restaurant"
"Restaurant" in venues_df['VenueCategory'].unique()

True

# Analyze Each Neighborhood


In [40]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(3650, 281)


,Neighborhoods,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Botanero,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Car Wash,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Classroom,College Football Field,College Gym,College Stadium,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Drive-in Theater,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Recreation Center,Recycling Facility,Residential Building (Apartment / Condo),Rest Area,Restaurant,Road,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Student Center,Supermarket,Sushi Restaurant,Swiss Restaurant,TV Station,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tex-Mex Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trade School,Trail,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Barrios Mágico

In [41]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(80, 281)


,Neighborhoods,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Botanero,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Car Wash,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Classroom,College Football Field,College Gym,College Stadium,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Drive-in Theater,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Recreation Center,Recycling Facility,Residential Building (Apartment / Condo),Rest Area,Restaurant,Road,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Student Center,Supermarket,Sushi Restaurant,Swiss Restaurant,TV Station,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tex-Mex Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trade School,Trail,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Barrio Chino (

In [42]:
len(kl_grouped[kl_grouped["Restaurant"] > 0])

55

# Create new data frame for Restauranta data only

In [45]:
kl_restaurant = kl_grouped[["Neighborhoods","Restaurant"]]

In [46]:
kl_restaurant.head()

,Neighborhoods,Restaurant
0,Barrio Chino (Mexico City),0.02
1,Barrios Mágicos of Mexico City,0.02
2,"Benito Juárez, Mexico City",0.00
3,Bosques de las Lomas,0.04
4,Centro (Mexico City),0.02


# Cluster Neighborhoods

In [48]:
#set number of clusters
kclusters = 4

kl_clustering = kl_restaurant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 0, 0, 3, 3, 3, 3, 3], dtype=int32)

In [52]:
kl_merged = kl_restaurant.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [53]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Barrio Chino (Mexico City),0.02,0
1,Barrios Mágicos of Mexico City,0.02,0
2,"Benito Juárez, Mexico City",0.00,3
3,Bosques de las Lomas,0.04,0
4,Centro (Mexico City),0.02,0


In [54]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(80, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Barrio Chino (Mexico City),0.02,0,19.442709,-99.100687
1,Barrios Mágicos of Mexico City,0.02,0,19.337676,-99.178496
2,"Benito Juárez, Mexico City",0.00,3,19.422300,-99.118520
3,Bosques de las Lomas,0.04,0,19.390980,-99.255120
4,Centro (Mexico City),0.02,0,19.431940,-99.133150


In [55]:
# sort results
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(80, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Barrio Chino (Mexico City),0.020000,0,19.442709,-99.100687
58,Pueblo Culhuacán,0.020000,0,19.336550,-99.107750
68,Santa María Tepepan,0.040000,0,19.274880,-99.136740
67,"Santa Fe, Mexico City",0.040000,0,19.379850,-99.244920
32,Colonia San Rafael,0.020000,0,19.439320,-99.162330
33,Colonia San Simón Tolnáhuac,0.020000,0,19.458637,-99.143284
35,Colonia Santa María la Ribera,0.020000,0,19.444911,-99.157690
36,Colonia Tabacalera,0.020000,0,19.435870,-99.152270
37,Colonia Tránsito,0.020000,0,19.400828,-99.186509
66,San Ángel,0.040000,0,19.346020,-99.191770


# Visualize Resulting clusters

In [56]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# Examine clusters

## Cluster 0

In [58]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Barrio Chino (Mexico City),0.020000,0,19.442709,-99.100687
58,Pueblo Culhuacán,0.020000,0,19.336550,-99.107750
68,Santa María Tepepan,0.040000,0,19.274880,-99.136740
67,"Santa Fe, Mexico City",0.040000,0,19.379850,-99.244920
32,Colonia San Rafael,0.020000,0,19.439320,-99.162330
33,Colonia San Simón Tolnáhuac,0.020000,0,19.458637,-99.143284
35,Colonia Santa María la Ribera,0.020000,0,19.444911,-99.157690
36,Colonia Tabacalera,0.020000,0,19.435870,-99.152270
37,Colonia Tránsito,0.020000,0,19.400828,-99.186509
66,San Ángel,0.040000,0,19.346020,-99.191770


## Cluster 1

In [59]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
63,San Miguel Chapultepec,0.060000,1,19.412450,-99.186750
51,Lomas de Chapultepec,0.060000,1,19.418773,-99.206001
75,"University City, Mexico",0.080000,1,19.357320,-99.260160
46,Jardines del Pedregal,0.080000,1,19.320940,-99.204560
24,Colonia Maza,0.080000,1,19.336789,-99.219128
42,Condesa,0.060000,1,19.415460,-99.176020
65,San Pedro Atocpan,0.068966,1,19.200620,-99.050070
11,Colonia Atlampa,0.060000,1,19.457220,-99.157890
12,Colonia Buenavista,0.064516,1,19.328920,-99.286818
34,Colonia Santa María Insurgentes,0.060000,1,19.461800,-99.152610


## Cluster 2

In [60]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
48,La Merced (neighborhood),0.2,2,18.8625,-99.625


## Cluster 3

In [61]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
76,Viaducto Piedad,0.0,3,19.402000,-99.132290
62,San Lorenzo Xochimanca,0.0,3,19.376530,-99.177430
71,Tacubaya,0.0,3,19.400600,-99.188280
70,"Tacuba, Mexico City",0.0,3,19.458890,-99.186590
64,San Miguel Topilejo,0.0,3,19.202000,-99.140850
39,Colonia Verónica Anzures,0.0,3,19.436600,-99.175150
59,San Andrés Mixquic,0.0,3,19.224250,-98.965350
2,"Benito Juárez, Mexico City",0.0,3,19.422300,-99.118520
5,Centro Urbano Benito Juárez,0.0,3,19.371800,-99.157250
6,Churubusco,0.0,3,19.355360,-99.150830


# Observations

Most of the restaurants are concentrated in the central area of Mexico city, with the highest number in cluster 0 and moderate number in cluster 1. On the other hand, cluster 2 has very low number to totally no restaurants in the neighborhoods. This represents a great opportunity and high potential areas to open new restaurants as there is very little to no competition from existing restaurants. Meanwhile, Restaurants in cluster 0 and 3 are likely suffering from intense competition due to oversupply and high concentration of other business. From another perspective, this also shows that the oversupply of restaurants mostly happened in the central area of the city, with the suburb area still have very few restaurants. Therefore, this project recommends property developers to capitalize on these findings to open new restaurants in neighborhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new restaurants in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 and 3 which already have high concentration of restaurants and suffering from intense competition.